<a href="https://colab.research.google.com/github/yash7125/makemore/blob/main/make_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-20 12:51:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-20 12:51:44 (16.6 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [41]:
with open('input.txt', 'r', encoding='utf-8') as p:
    text = p.read()

In [42]:
len(text)

1115394

In [43]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [44]:
# tokenise your data
chars = sorted(list(set(text)))
print(len(chars))

str = ""
for i in range(len(chars)):
  str += chars[i]

print(str)

vocab_size = 65


65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [45]:
# do encoding and decoding
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

def encode(s):
  enc = []
  for i in range(len(s)):
    enc.append(stoi[s[i]])
  return enc

def decode(l):
  str = ""
  for i in range(len(l)):
    str += itos[l[i]]
  return str

encoder = encode("Yash Kumar")
decoder = decode(encoder)

print(encoder, " ", decoder)

[37, 39, 57, 46, 1, 23, 59, 51, 39, 56]   Yash Kumar


In [8]:
# now encode the entire data and store it in tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [46]:
# create train and val set
length = int(0.9*len(data))
# print(length)
train_set = data[:length]
val_set = data[length:]

In [47]:
block_size = 8
train_set[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [48]:
Xtr = train_set[:block_size]
Ytr = train_set[1 : block_size + 1]

for i in range(block_size):
  x = Xtr[:i + 1]
  y = Ytr[i]
  print(f"we have the context window {x} --> target --> {y}")


we have the context window tensor([18]) --> target --> 47
we have the context window tensor([18, 47]) --> target --> 56
we have the context window tensor([18, 47, 56]) --> target --> 57
we have the context window tensor([18, 47, 56, 57]) --> target --> 58
we have the context window tensor([18, 47, 56, 57, 58]) --> target --> 1
we have the context window tensor([18, 47, 56, 57, 58,  1]) --> target --> 15
we have the context window tensor([18, 47, 56, 57, 58,  1, 15]) --> target --> 47
we have the context window tensor([18, 47, 56, 57, 58,  1, 15, 47]) --> target --> 58


In [49]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_batch(split):
  data = train_set if split == 'train' else val_set
  idx = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i + block_size] for i in idx])
  y = torch.stack([data[i + 1 : i + block_size + 1] for i in idx])
  x, y = x.to(device), y.to(device)
  return x, y

Xb, Yb = get_batch('train')
print(Xb.shape)
print(Xb)

print(Yb.shape)
print(Yb)

print('\n')

for i in range(batch_size):
  for j in range(block_size):
    p = Xb[i, :j + 1]
    q = Yb[i, j]
    print(f"we have the context window {p} --> target --> {q}")



torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


we have the context window tensor([24]) --> target --> 43
we have the context window tensor([24, 43]) --> target --> 58
we have the context window tensor([24, 43, 58]) --> target --> 5
we have the context window tensor([24, 43, 58,  5]) --> target --> 57
we have the context window tensor([24, 43, 58,  5, 57]) --> target --> 1
we have the context window tensor([24, 43, 58,  5, 57,  1]) --> target --> 46
we have the context window tensor([24, 43, 58,  5, 57,  1, 46]) --> target --> 43
we have the context window tensor([24, 43, 58,  5, 57,  1, 46, 43]) --> target --> 39
we have the context window tensor([44]) --> targ

In [50]:
# # bigram model
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# class BigramLanguageModel(nn.Module):

#   def __init__(self, vocab_size):
#     super().__init__()
#     self.token_embedding_table = torch.nn.Embedding(vocab_size, vocab_size)

#   def forward(self, idx, targets=None):
#     # idx --> [B, T], Embedding_table --> [65, 65]
#     logits = self.token_embedding_table(idx)

#     if targets == None:
#       loss = None
#     else:
#       # logits --> [B, T, 65] --> [B, T, C]
#       # reshaping to solve cross entropy
#       B, T, C = logits.shape
#       logits = logits.view(B*T, C)
#       targets = targets.view(B*T)
#       loss = torch.nn.functional.cross_entropy(logits, targets)

#     return logits, loss

#   def generate(self, idx, max_new_tokens):

#     for _ in range(max_new_tokens):

#       logits, loss = self(idx)
#       # logits --> [32, 65] --> [B*T, C]
#       # print("logits shape before = ", logits.shape)
#       logits = logits[:, -1, :]
#       # logits --> [4, 65] --> [B, C]
#       # print("logits shape after = ", logits.shape)

#       probs = F.softmax(logits, dim=-1)
#       # probs --> [B, C]
#       idx_next = torch.multinomial(probs, num_samples=1)
#       # idx_next --> [B, 1]
#       idx = torch.cat((idx, idx_next), dim=1)
#       # idx --> [B, T] --> [B, T+1]
#     return idx


# ob = BigramLanguageModel(vocab_size)
# logits, loss = ob(Xb, Yb)
# print(logits.shape, " ", "loss =", loss)
# # print(logits[:5])

# print(decode(ob.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))




In [51]:
# optimizer = torch.optim.AdamW(ob.parameters(), lr=1e-3)

In [52]:
# # do the training of bigram model

# for i in range(20000):
#   # batch declaration
#   xb, yb = get_batch('train')
#   # forward pass
#   logits, loss = ob(xb, yb)
#   # print(i, "->", loss)
#   # setting gradients to None
#   optimizer.zero_grad(set_to_none=True)
#   # back prop
#   loss.backward()
#   # parameters update
#   optimizer.step()

# print(loss.item())

In [53]:
# print(decode(ob.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

In [54]:
# # now comes the self-attention
# torch.manual_seed(1337)
# B = 4
# T = 8
# C = 2
# x = torch.randn(B, T, C)

In [55]:
# #version-1
# xbow = torch.zeros((B, T, C))
# for b in range(B):
#   for t in range(T):
#     x_prev = x[b, :t + 1] #(t, C)
#     # print(x_prev.shape)
#     xbow[b, t] = torch.mean(x_prev, 0)


In [56]:
# xbow.shape
# xbow

In [57]:
# # version-2
# wei = torch.tril(torch.ones(T, T))
# wei = wei / wei.sum(1, keepdim=True)
# wei
# xbow2 = wei @ x #(T,T) multiply with (B,T,C) --> (B,T,T) @ (B,T,C) --> (B,T,C)
# torch.allclose(xbow, xbow2)
# # xbow2

In [58]:
# # version-3 using softmax
# tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
# wei = wei.masked_fill(tril==0, float('-inf'))
# wei = F.softmax(wei, dim=-1)
# xbow3 = wei @ x
# torch.allclose(xbow, xbow3)

In [59]:
# xbow3

In [60]:
# # now starting to implement self-attention
# # version-4
# torch.manual_seed(1337)
# B, T, C = 4, 8, 32
# x = torch.randn(B, T, C)

# # attention is communication mechanism
# head_size = 16

# query = nn.Linear(C, head_size, bias=False) #(C, 16) --> (32, 16)
# key = nn.Linear(C, head_size, bias=False)   #(C, 16) --> (32, 16)
# value = nn.Linear(C, head_size, bias=False) #(C, 16) --> (32, 16)

# q = query(x) #(B, T, 16) --> (4, 8, 16)
# k = key(x)   #(B, T, 16) --> (4, 8, 16)
# v = value(x) #(B, T, 16) --> (4, 8, 16)

# wei = q @ k.transpose(-2, -1) #(B, T, T) --> (4, 8, 8)
# tril = torch.tril(torch.ones(T, T))
# wei = wei.masked_fill(tril==0, float('-inf')) #(current value should not interact with future predictions)
# wei = F.softmax(wei, dim=-1)
# out = wei @ v
# out.shape

In [61]:
# out[2], wei[2]

In [62]:
# more elaborate code from the beginning

In [76]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
vocab_size = 65
n_embd = 200
block_size = 64 #(maximum context window length)
batch_size = 16 #(how many independent sequences will run parallel?)
# head_size = 16
eval_iters = 500
n_layers = 6
num_of_heads = 5
max_iters = 5000
eval_interval = 100
learning_rate = 9e-4
dropout = 0.2

torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):

  def __init__(self, head_size):
    # attention is communication mechanism
    super().__init__()
    self.query = nn.Linear(n_embd, head_size, bias=False) #(n_embd, head_size) --> (64, 16)
    self.key = nn.Linear(n_embd, head_size, bias=False)   #(n_embd, head_size) --> (64, 16)
    self.value = nn.Linear(n_embd, head_size, bias=False) #(n_embd, head_size) --> (64, 16)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):

    B, T, C = x.shape

    q = self.query(x) #(B, T, 16) --> (16, T, 16)
    k = self.key(x)   #(B, T, 16) --> (16, T, 16)
    v = self.value(x) #(B, T, 16) --> (16, T, 16)

    wei = q @ k.transpose(-2, -1) * (C ** -0.5) #(B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) #(current value should not interact with future predictions)
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    out = wei @ v

    return out

class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):

    super().__init__()

    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):

    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.proj(out)
    out = self.dropout(out)

    return out

class FeedForward(nn.Module):

  def __init__(self, n_embd):

    super().__init__()

    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),

        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

# class LayerNorm1d: # (used to be BatchNorm1d)

#   def __init__(self, dim, eps=1e-5, momentum=0.1):
#     self.eps = eps
#     self.gamma = torch.ones(dim)
#     self.beta = torch.zeros(dim)

#   def __call__(self, x):
#     # calculate the forward pass
#     xmean = x.mean(1, keepdim=True) # batch mean
#     xvar = x.var(1, keepdim=True) # batch variance
#     xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
#     self.out = self.gamma * xhat + self.beta
#     return self.out

#   def parameters(self):
#     return [self.gamma, self.beta]


class Block(nn.Module):
  # Transformer block --> communication followed by computation

  def __init__(self, n_embd, n_heads):

    super().__init__()

    head_size = n_embd // n_heads

    self.mha = MultiHeadAttention(n_heads, head_size)
    self.ffn = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):

    x = x + self.mha(self.ln1(x))
    x = x + self.ffn(self.ln2(x))

    return x


class BigramLanguageModel(nn.Module):

  def __init__(self):

    super().__init__()

    self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embd)    #(65, 64)
    self.position_embedding_table = torch.nn.Embedding(block_size, n_embd) #(32, 64)
    self.blocks = nn.Sequential(
        *[Block(n_embd, n_heads=num_of_heads) for _ in range(n_layers)]
    )
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    # idx --> [B, T], Embedding_table --> [65, 64]
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))

    x = tok_emb + pos_emb #(B, T, n_embd)
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)   #(B, T, vocab_size)

    if targets == None:
      loss = None
    else:
      # logits --> [B, T, 65] --> [B, T, C]
      # reshaping to solve cross entropy
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = torch.nn.functional.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):

      idx_sec = idx[:, -block_size:]
      logits, loss = self(idx_sec)
      # logits --> [32, 65] --> [B*T, C]
      logits = logits[:, -1, :]
      # logits --> [4, 65] --> [B, C]
      probs = F.softmax(logits, dim=-1)
      # probs --> [B, C]
      idx_next = torch.multinomial(probs, num_samples=1)
      # idx_next --> [B, 1]
      idx = torch.cat((idx, idx_next), dim=1)
      # idx --> [B, T] --> [B, T+1]
    return idx



In [77]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()




2.931265 M parameters
step 0: train loss 4.2931, val loss 4.2938
step 100: train loss 2.4981, val loss 2.4981
step 200: train loss 2.3586, val loss 2.3663
step 300: train loss 2.2283, val loss 2.2496
step 400: train loss 2.1265, val loss 2.1640
step 500: train loss 2.0542, val loss 2.1039
step 600: train loss 1.9882, val loss 2.0638
step 700: train loss 1.9254, val loss 2.0220
step 800: train loss 1.8873, val loss 1.9867
step 900: train loss 1.8419, val loss 1.9512
step 1000: train loss 1.8069, val loss 1.9452
step 1100: train loss 1.7664, val loss 1.8992
step 1200: train loss 1.7447, val loss 1.8967
step 1300: train loss 1.7264, val loss 1.8791
step 1400: train loss 1.7030, val loss 1.8597
step 1500: train loss 1.6760, val loss 1.8304
step 1600: train loss 1.6727, val loss 1.8313
step 1700: train loss 1.6565, val loss 1.8246
step 1800: train loss 1.6397, val loss 1.8040
step 1900: train loss 1.6316, val loss 1.8001
step 2000: train loss 1.6117, val loss 1.7884
step 2100: train loss 1.

In [78]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx=context, max_new_tokens=1000)[0].tolist()))


He hope's and penla thereft,
Hark watch'd as to inswer me the ported
Tongue, seem while's infand their eyes Keeps exroard
What fight swife, he's a trifield; or oaten to tawe,
What this speeir, benoughtly it of a plaison!
Unry Angelofory you like a fiarm
Than offate as with fortage, my graveford,
Or in your sparking yone, count which
how is Hy coun, indertnous gentle fair.

LUCE:
The dust thattern saidly, your will widow's iso righter.

LADY CAPULIS:
He's gother!
Why wayce, canno not-morroase house; and my and and were as Lord
Bettake give with him.' They tell you need of at ioon,
May that I deserver you, princhid evere that do punil
In prince Edward againful. Yet shall my court.

PRINCE:
Ay, thou well wants and trump, my lifes if young. Why, resolus.

DUKE VINCENTIO:
For it. Ye', you dare it be poor mistry;
And save I diegdom othad.

DUKE VINCENTIO:
That all you snoursel: our eyes; wast, deservingdom
Call'd with him. Yesry; do welce: if this be
Comist mameger have surmersedgethed
By P